In [4]:
import torch
import math
from torch import nn, save, load
from torchvision import datasets

# Load the model.pt file
PATH = "model.pt"
model_params_dict = torch.load(PATH,map_location=torch.device('cpu'))
model_params = model_params_dict["model_state_dict"]

<ipython-input-4-8e63d6084e7f>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_params_dict = torch.load(PATH,map_location=torch.device('cpu'))


In [5]:
model_params.keys()

odict_keys(['model.0.weight', 'model.0.bias', 'model.4.weight', 'model.4.bias'])

In [29]:
# _______________________GENERAL FLOW___________________
# primitives stay constant
# On the Vitis side, merge the primitives and arch for synthesis master file

#________________________WRITE_OUT_________________________
# write the model arch file

arch_name = "bare"
arch_file = open("{}_model_syn.cpp".format(arch_name),'w')

arch_file.write("// ___________________________________________CNN_SYN BEGINS__________________________________________________________\n")

arch_file.write("void {}_model ( data_t* test_images, \n".format(arch_name))
arch_file.write("\t data_t* preds, \n")

for i,(name, param) in enumerate(model_params.items()):
    idx = int(math.floor(i/2 + 1))

    if (len(param.shape)==4):
        arch_file.write("\t data_t* fp_conv_{}_weights, \n".format(idx))
        prev_flag = "conv"
    if (len(param.shape)==2):
        arch_file.write("\t data_t* fp_dense_{}_weights, \n".format(idx))
        prev_flag = "dense"
    if (len(param.shape)==1):
        if (prev_flag=="conv"):
            arch_file.write("\t data_t* fp_conv_{}_bias, \n".format(idx))
        if (prev_flag=="dense"):
            arch_file.write("\t data_t* fp_dense_{}_bias, \n".format(idx))

arch_file.write("\t ) \n")


5

In [30]:
arch_file.write("\n{ \n")
arch_file.write("//_________________________________PULL_DATA___________________________\n")

for i,(name, param) in enumerate(model_params.items()):
    idx = int(math.floor(i/2 + 1))

    if len(param.shape) == 4:  # For Convolutional Layer Weights
        arch_file.write(f"\tdata_t fp_CONV_{idx}_WEIGHTS [CONV_{idx}_WEIGHTS_N * CONV_{idx}_WEIGHTS_C * CONV_{idx}_WEIGHTS_H * CONV_{idx}_WEIGHTS_W];\n")
        arch_file.write(f"\tp_conv{idx}_w_loop: for (int i=0; i < CONV_{idx}_WEIGHTS_N * CONV_{idx}_WEIGHTS_C * CONV_{idx}_WEIGHTS_H * CONV_{idx}_WEIGHTS_W; i++){{\n")
        arch_file.write(f"\t\tfp_CONV_{idx}_WEIGHTS[i] = fp_conv_{idx}_weights[i];\n")
        arch_file.write("\t}\n")
        prev_flag = "conv"

    if len(param.shape) == 2:  # For Dense Layer Weights
        arch_file.write(f"\tdata_t fp_DENSE_{idx}_WEIGHTS [DENSE_{idx}_WEIGHTS_H * DENSE_{idx}_WEIGHTS_W];\n")
        arch_file.write(f"\tp_dense{idx}_w_loop: for (int i=0; i < DENSE_{idx}_WEIGHTS_H * DENSE_{idx}_WEIGHTS_W; i++){{\n")
        arch_file.write(f"\t\tfp_DENSE_{idx}_WEIGHTS[i] = fp_dense_{idx}_weights[i];\n")
        arch_file.write("\t}\n")
        prev_flag = "dense"

    if len(param.shape) == 1:  # For Biases
        if prev_flag == "conv":
            arch_file.write(f"\tdata_t fp_CONV_{idx}_BIAS [CONV_{idx}_BIAS_N];\n")
            arch_file.write(f"\tp_conv{idx}_b_loop: for (int i=0; i < CONV_{idx}_BIAS_N; i++){{\n")
            arch_file.write(f"\t\tfp_CONV_{idx}_BIAS[i] = fp_conv_{idx}_bias[i];\n")
            arch_file.write("\t}\n")
        elif prev_flag == "dense":
            arch_file.write(f"\tdata_t fp_DENSE_{idx}_BIAS [DENSE_{idx}_BIAS_N];\n")
            arch_file.write(f"\tp_dense{idx}_b_loop: for (int i=0; i < DENSE_{idx}_BIAS_N; i++){{\n")
            arch_file.write(f"\t\tfp_DENSE_{idx}_BIAS[i] = fp_dense_{idx}_bias[i];\n")
            arch_file.write("\t}\n")

arch_file.write(f"\tdata_t fp_TEST_IMGS [BATCH_SIZE * NUM_CHANNELS * IMG_HEIGHT * IMG_WIDTH];\n")
arch_file.write(f"\tp_test_data_loop: for (int i=0; i < BATCH_SIZE * NUM_CHANNELS * IMG_HEIGHT * IMG_WIDTH; i++){{\n")
arch_file.write(f"\t\tfp_TEST_IMGS[i] = test_images[i];\n")
arch_file.write("\t}\n")

arch_file.write("\n}\n")
arch_file.close()